# Импорт библиотек

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32884 sha256=c485d59877a1c35b006ba72db34689dcbdb2bba343690b511d940706471bbe90
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [3]:
!pip install razdel

In [4]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 8.2MB 12.8MB/s 


In [6]:
!cp "/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/src/preprocessing_tools/preprocessing_tools.py" .

In [7]:
!cp "/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/src/preprocessing_tools/stopwords.txt" .

In [8]:
import pandas as pd
from razdel import tokenize 
import re
import preprocessing_tools as pr

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Получение данных из РПД

пример извлечения текста из документа


In [9]:
prep_tools = pr.preprocessing_tools()

In [10]:
file_data= pr.parser.from_file("/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/18_056866_Введение в блокчейн технологии_База.pdf")
text = file_data['content']
print(text)

2020-11-17 08:00:05,616 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-11-17 08:00:06,366 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-11-17 08:00:06,845 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...




















































  
  
  
  
  
  
  
  
  
  
  
  
  
  

Санкт-Петербургский государственный университет  
 
  
 
 

Р А Б О Ч А Я   П Р О Г Р А М М А  
УЧЕБНОЙ ДИСЦИПЛИНЫ  

 
 

Введение в блокчейн технологии  
An Introduction to Blockchain Technologies 

 
 

Язык(и) обучения 
  

русский 
 
 

Трудоемкость в зачетных единицах: 3 
  

Регистрационный номер рабочей программы: 056866 
  

  
  



Раздел 1. Характеристики учебных занятий 
 
1.1. Цели и задачи учебных занятий 
Основной целью освоения дисциплины «Введение в блокчейн технологии» является 
приобретение обучающимися знаний об основных понятиях, технологиях и способах 
применения технологии блокчейн, а также практических навыков работы с 
инструментами, применяемыми в области криптовалют. 
Поставленная цель достигается путем решения следующих задач курса: 
1) Ознакомить студентов с основными задачами, решаемыми в области криптовалют, 
базовыми алгоритмами этой области, а также со сферами 

Получение текстов из документов


In [11]:
RPD_texts = prep_tools.extract_text_from_pdfs_recursively('/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD')

Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001121_ Введение в задачи исследования систем.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001114_Современные СУБД.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001111_Современная философия и методология науки.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001115_Научно-исследовательская работа (курсовая работа).pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001113_Объектно-ориентированные CASE технологии.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_sem

Преобразование текстов в датафрейм


In [12]:
df = pd.DataFrame(RPD_texts, columns=['raw_text'], dtype='str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raw_text  40 non-null     object
dtypes: object(1)
memory usage: 448.0+ bytes


In [13]:
df.head()

,raw_text
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


# Векторизация текстов

In [14]:
df['clean_text'] = df['raw_text'].apply(prep_tools.clean_text)
df['clean_text_no_stopwords'] = df['clean_text'].apply(prep_tools.stop_words_remove)
df.head()

,raw_text,clean_text,clean_text_no_stopwords
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,санктпетербургский государственный университет...,"['санктпетербургский', 'государственный', 'уни..."
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а учебной дис...,"['учебный', 'дисциплина', 'современный', 'субд..."
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,санктпетербургский государственный университет...,"['санктпетербургский', 'государственный', 'уни..."
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,санктпетербургский государственный университет...,"['санктпетербургский', 'государственный', 'уни..."
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а учебной дис...,"['учебный', 'дисциплина', 'объектноориентирова..."


In [15]:
 def preprocessing(text):
     cleared_text = prep_tools.clean_text(text)
     clean_text_without_stopwords = prep_tools.stop_words_remove(cleared_text)
     return clean_text_without_stopwords

In [16]:
def preprocessing_1(text):
     cleared_text = prep_tools.clean_text(text) # очистка текста
     clean_text_without_stopwords = prep_tools.lemmatization(cleared_text) # токенизация, лемматизация, удаление стоп-слов
     return clean_text_without_stopwords

In [17]:
test_text = "Значимость этих проблем настолько очевидна, что консультация с широким активом играет важную роль в формировании позиций, занимаемых участниками в отношении поставленных задач. Разнообразный и богатый опыт новая модель организационной деятельности позволяет оценить значение дальнейших направлений развития. Идейные соображения высшего порядка, а также дальнейшее развитие различных форм деятельности представляет собой интересный эксперимент проверки дальнейших направлений развития."

In [18]:
print(preprocessing(test_text))

['значимость', 'проблема', 'настолько', 'очевидный', 'консультация', 'широкий', 'актив', 'играть', 'важный', 'роль', 'формирование', 'позиция', 'занимать', 'участник', 'отношение', 'поставить', 'задача', 'разнообразный', 'богатый', 'опыт', 'новый', 'модель', 'организационный', 'деятельность', 'позволять', 'оценить', 'значение', 'дальнейший', 'направление', 'развитие', 'идейный', 'соображение', 'высокий', 'порядок', 'также', 'дальнейший', 'развитие', 'различный', 'форма', 'деятельность', 'представлять', 'интересный', 'эксперимент', 'проверка', 'дальнейший', 'направление', 'развитие']


In [21]:
print(preprocessing_1(test_text))

['значимость', 'проблема', 'настолько', 'очевидный', 'консультация', 'широкий', 'актив', 'играть', 'важный', 'роль', 'формирование', 'позиция', 'занимать', 'участник', 'отношение', 'поставить', 'задача', 'разнообразный', 'богатый', 'опыт', 'новый', 'модель', 'организационный', 'деятельность', 'позволять', 'оценить', 'значение', 'дальнейший', 'направление', 'развитие', 'идейный', 'соображение', 'высокий', 'порядок', 'также', 'дальнейший', 'развитие', 'различный', 'форма', 'деятельность', 'представлять', 'интересный', 'эксперимент', 'проверка', 'дальнейший', 'направление', 'развитие']


In [19]:
vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1, 2), preprocessor=preprocessing)
Vectorized_RPD = vectorizer.fit_transform(df.raw_text)
Vectorized_RPD

<40x29241 sparse matrix of type '<class 'numpy.float64'>'
	with 71840 stored elements in Compressed Sparse Row format>

In [20]:
vectorizer.get_feature_names()

['aa',
 'aa произвольный',
 'aa равный',
 'aah',
 'aah помощь',
 'aah считать',
 'ability',
 'ability to',
 'able',
 'able to',
 'about',
 'about highly',
 'about how',
 'about the',
 'about transmitting',
 'academic',
 'academic module',
 'academic project',
 'academic purposes',
 'academic vocabulary',
 'academic writing',
 'access',
 'access to',
 'according',
 'according to',
 'accuracy',
 'accuracy of',
 'acid',
 'acid влияние',
 'acm',
 'acm digital',
 'acquisition',
 'acquisition processing',
 'acrobat',
 'acrobat reader',
 'actian',
 'actian matrix',
 'action',
 'action furthermore',
 'action in',
 'action or',
 'action temporary',
 'action trend',
 'action which',
 'actions',
 'actions in',
 'actions speak',
 'actions which',
 'active',
 'active links',
 'activities',
 'activities operations',
 'activity',
 'activity monitor',
 'actually',
 'actually simpler',
 'actually they',
 'adaboost',
 'adaboost бэггинга',
 'adaboost стохастический',
 'adam',
 'adam shook',
 'adaptation'